In [6]:
!pip install tensorflow  
!pip install pandas 
!pip install matplotlib
!pip install scikit-learn

In [7]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [8]:
df=pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv','train.csv'))

In [9]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
#Pre process the data   

In [11]:
from tensorflow.keras.layers import TextVectorization

In [12]:
max_words=200000

In [13]:
vectorizer=TextVectorization(max_tokens=max_words,output_sequence_length=1800,output_mode='int')

In [14]:
x=df['comment_text']

In [15]:
y=df[df.columns[2:]].values

In [16]:
vectorizer.adapt(x.values)

In [17]:
vectorizer_text=vectorizer(x.values)

In [18]:
#vectorizer_text

In [19]:
vectorizer("Lumicrystal as")[:10]

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 1, 18,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int64)>

In [20]:
#test=vectorizer.get_vocabulary()

In [21]:
#"stratophere" in test

In [22]:
dataset=tf.data.Dataset.from_tensor_slices((vectorizer_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [23]:
# dataset.as_numpy_iterator().next()

In [24]:
train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [25]:
# train.as_numpy_iterator().next()

In [26]:
#sequential model

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM ,Dense ,Embedding, Dropout,Bidirectional

In [28]:
model=Sequential()
model.add(Embedding(max_words+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))

model.add(Dense(128,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(128,activation="relu"))

model.add(Dense(6,activation="sigmoid"))


In [29]:
model.compile(loss="BinaryCrossentropy", optimizer="adam")

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [31]:
history=model.fit(train,epochs=1,validation_data=val)

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
#prediction

In [54]:
input_str=vectorizer("i will find u ,and kill u")

In [55]:
model.predict(np.expand_dims(input_str,0)) >0.5

1/1 [==============================] - 0s 161ms/step


array([[ True, False,  True, False,  True, False]])

In [59]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [62]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 184ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [63]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [ ]:
#evaluate

In [64]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [65]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [67]:
for batch in test.as_numpy_iterator(): 
    X_true, y_true = batch
    yhat = model.predict(X_true)
    
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 172ms/step


KeyboardInterrupt: 

In [68]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.9122806787490845, Recall:0.7647058963775635, Accuracy:0.5


In [69]:
#Appp

In [70]:
!pip install gradio jinja2

In [71]:
import tensorflow as tf
import gradio as gr

c:\Users\Lenovo\Desktop\Project\kesar1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model.save('test_toxicity.h5')

In [ ]:
model = tf.keras.models.load_model('test_toxicity.h5')

In [72]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [75]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [76]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://90c3a64758fbbc95c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 83ms/step
